In [ ]:
import numpy
import pickle

In [ ]:
from snippets.ns_helper import cavity_flow, velocity_term

In [ ]:
def run_cavity():
    nx = 41
    ny = 41
    with open('IC.pickle', 'rb') as f:
        u, v, p, b = pickle.load(f)

    dx = 2 / (nx - 1)
    dy = 2 / (ny - 1)
    dt = .005
    nt = 500
    
    u, v, p = cavity_flow(u, v, p, nt, dt, dx, 
                         velocity_term, 
                         pressure_poisson, 
                         rtol=1e-4)
    
    return u, v, p

In [ ]:
with open('numpy_ans.pickle', 'rb') as f:
    u, v, p = pickle.load(f)

## Other options for accelerating Python code

## Cython

In [ ]:
%load_ext cython

In [ ]:
%%cython
cimport numpy
cimport cython

import numpy

from libc.math cimport sqrt

@cython.boundscheck(False)
def pressure_poisson(numpy.ndarray[numpy.float_t, ndim=2] p,
                     numpy.ndarray[numpy.float_t, ndim=2] b,
                     double l2_target):

    cdef numpy.ndarray[numpy.float_t, ndim=2] pn = numpy.zeros_like(p)
    cdef int j, i, n
    cdef double iter_diff
    cdef int J = b.shape[0]
    cdef int I = b.shape[1]

    iter_diff = l2_target + 1

    n = 0
    while iter_diff > l2_target:
        pn = p.copy()
        for j in range(1, J - 1):
            for i in range(1, I - 1):
                p[j, i] = (.25 * (pn[j, i + 1] +
                                  pn[j, i - 1] +
                                  pn[j + 1, i] +
                                  pn[j - 1, i]) -
                                  b[j, i])

        for j in range(J):
            p[j, 0] = p[j, 1]
            p[j, -1] = p[j, -2]

        for i in range(I):
            p[0, i] = p[1, i]
            p[-1, i] = 0

        if n % 10 == 0:
            iter_diff = L2_Cython(p, pn)
        if n == 500:
            break
        n += 1

    return p


@cython.boundscheck(False)
def L2_Cython(numpy.ndarray[numpy.float_t, ndim=2] p,
              numpy.ndarray[numpy.float_t, ndim=2] pn):
    cdef double error = 0
    cdef double pnsum = 0
    cdef int ny = p.shape[0]
    cdef int nx = p.shape[1]
    for i in range(nx):
        for j in range(ny):
            error += (p[i,j]-pn[i,j])**2
            pnsum += pn[i,j]**2

    return sqrt(error/pnsum)


In [ ]:
%timeit run_cavity()

In [ ]:
uc, vc, pc = run_cavity()
assert numpy.allclose(u, uc)
assert numpy.allclose(v, vc)
assert numpy.allclose(p, pc)

## Fortran and `f2py`

In [ ]:
%%file pressure_poisson_F.f90

SUBROUTINE pressure_poisson(p, b, N, M, l2_target)
IMPLICIT NONE

INTEGER(4), INTENT(IN):: N, M
REAL(8), INTENT(IN):: b(N, M), l2_target
REAL(8), INTENT(INOUT):: p(N, M)
REAL(8):: iter_diff, pn(N, M)
INTEGER(4):: c

!F2PY intent(inout):: b
!F2PY intent(inplace, out):: p
!F2PY real(8), optional, intent(in):: l2_target=1E-3
!F2PY integer(4), intent(hide), depend(p):: n=shape(p, 0), m=shape(p, 1)

    c = 0
    iter_diff = l2_target + 1
    
    DO WHILE(iter_diff > l2_target)
        pn = p
        p(2:N-1, 2:M-1) = .25 * (pn(2:N-1, 3:M) + pn(2:N-1, 1:M-2) + &
                                 pn(3:N, 2:M-1) + pn(1:N-2, 2:M-1)) - b(2:N-1, 2:M-1)
        
        p(1:N, 1) = p(1:N, 2)
        p(1:N, M) = p(1:N, M-1)
        p(1, 1:M) = p(2, 1:M)
        p(N, 1:M) = 0
        
        IF (MOD(c, 10) .eq. 0) iter_diff = DSQRT(SUM((p - pn)**2)/SUM(pn**2))
        IF (c .eq. 500) EXIT
        
        c = c + 1
    ENDDO
    
END SUBROUTINE pressure_poisson

In [ ]:
!f2py3 -c --fcompiler=gnu95 \
      --f90flags= --f77flags= --opt="-m64 -O4" \
      -m pressure_poisson_F pressure_poisson_F.f90 \
      -DF2PY_REPORT_ON_ARRAY_COPY=1 \
      > /dev/null

In [ ]:
from pressure_poisson_F import pressure_poisson

In [ ]:
%timeit run_cavity()

In [ ]:
uf, vf, pf = run_cavity()
assert numpy.allclose(u, uf)
assert numpy.allclose(v, vf)
assert numpy.allclose(p, pf)